<a href="https://colab.research.google.com/github/shreya1110-dev/IBM-Project-27630-1660061379/blob/main/tulu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Oct 25 08:57:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 7.6 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3163375 sha256=0dd820fad1a15e8a315a48c1bf9b63f36db4f2f8cd55ac4f41b86c8dc45a10ae
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np 
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
import string
import warnings
import fasttext
import fasttext.util
import gensim 
from nltk.tokenize import sent_tokenize, word_tokenize 
from numpy import array
from numpy import asarray
from numpy import zeros
from gensim.models import Word2Vec 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_validate
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

Mounted at /content/drive


In [4]:
warnings.filterwarnings('ignore')

In [5]:
refined_data = pd.read_excel("/content/drive/MyDrive/fire/tulu-text-refined.xlsx")

In [6]:
sentiment_data = refined_data[["Comment", 	"Sentiment: Positive, neagtive, neutral"]]
sentiment_data

,Comment,"Sentiment: Positive, neagtive, neutral"
0,Excellent...vaa malla malla show la umdeta dum...,P
1,Valerie sir na voice keniji YouTube de,N
2,Firstrde bloopers thojale plz,NU
3,Congress 100 episode mantinek niklna mata epi...,P
4,Err 2 jana matra aath....full team malthini ep...,N
...,...,...
1694,Congratulations both of u.. ❤️chende da uncle ...,NU
1695,Teacher na principal Yana athige,NU
1696,Dala unda... song launch (programme) dha video...,NU
1697,Sachin anna ❤️\nDala unda... 🍻,NU


In [7]:
sentiment_data['Sentiment: Positive, neagtive, neutral'] = sentiment_data['Sentiment: Positive, neagtive, neutral'].replace((" NU","NU "),("NU","NU"))
sentiment_data['Sentiment: Positive, neagtive, neutral'] = sentiment_data['Sentiment: Positive, neagtive, neutral'].replace((" N","N "),("N","N"))
sentiment_data['Sentiment: Positive, neagtive, neutral'] = sentiment_data['Sentiment: Positive, neagtive, neutral'].replace((" P","P "," p","p ","p"),("P","P","P","P","P"))
sentiment_data.groupby(by=['Sentiment: Positive, neagtive, neutral'], dropna=False).count()

,Comment
"Sentiment: Positive, neagtive, neutral",
N,126
NU,906
P,667


In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [10]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
X = sentiment_data['Comment']
Y = sentiment_data['Sentiment: Positive, neagtive, neutral']

In [12]:
X_final=[]
for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ', X.iloc[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    X_final.append(review)
sentiment_data['Clean'] = X_final

In [39]:
# create header
head = ["Metric", "Value"]
 
def cross_validation(model, _X, _y, _cv=5):
      _scoring = ['precision_macro', 'recall_macro', 'balanced_accuracy', 'f1_weighted']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      # assign data
      # data = {"Mean Training Accuracy":results['train_balanced_accuracy'].mean()*100,              
      #         "Mean Training Precision": results['train_precision_macro'].mean()*100,
      #         "Mean Training Recall": results['train_recall_macro'].mean()*100,
      #         "Mean Training F1 Score": results['train_f1_weighted'].mean()*100,
      #         "Mean Validation Accuracy": results['test_balanced_accuracy'].mean()*100,
      #         "Mean Validation Precision": results['test_precision_macro'].mean()*100,
      #         "Mean Validation Recall": results['test_recall_macro'].mean()*100,
      #         "Mean Validation F1 Score": results['test_f1_weighted'].mean()*100,
      #         "Training Accuracy": results['train_balanced_accuracy'],              
      #         "Training Precision": results['train_precision_macro'],
      #         "Training Recall": results['train_recall_macro'],
      #         "Training F1 Score": results['train_f1_weighted'],
      #         "Validation Accuracy": results['test_balanced_accuracy'],
      #         "Validation Precision": results['test_precision_macro'],
      #         "Validation Recall": results['test_recall_macro'],
      #         "Validation F1 Score": results['test_f1_weighted']
      # }
      # display table
      #print(tabulate(data, headers=head, tablefmt="grid"))
      f1_scores.append(results['test_f1_weighted'].mean()*100)
      acc_scores.append(results['test_balanced_accuracy'].mean()*100)

In [20]:
def evaluate_model(model, X, Y):
  cross_validation(model, X, Y, 5)

  # plt.bar(['Fold 1','Fold 2','Fold 3','Fold 4','Fold 5'], scores, color ='blue',width = 0.4)
  # plt.show()

In [21]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_tf = tf_idf.fit_transform(X_final)

In [16]:
# print("n_samples: %d, n_features: %d" % X_tf.shape)

In [40]:
f1_df = pd.DataFrame()
f1_df.insert(loc=0,
          column='Model',
          value=['Naive Bayes', 'KNN Classifier', 'SVC', 'Decision tree', 'Adaboost', 'GNB', 
                 'Logistic regression', 'MLP', 'LDA', 'Random forest'])

acc_df = pd.DataFrame()
acc_df.insert(loc=0,
          column='Model',
          value=['Naive Bayes', 'KNN Classifier', 'SVC', 'Decision tree', 'Adaboost', 'GNB', 
                 'Logistic regression', 'MLP', 'LDA', 'Random forest'])

In [41]:
f1_scores = []
acc_scores = []

#naive bayes classifier
print("NAIVE BAYES CLASSIFIER")
naive_bayes_classifier = MultinomialNB()
evaluate_model(naive_bayes_classifier, X_tf, Y)

# knn classifier
print("KNN CLASSIFIER")
knn_classifier = KNeighborsClassifier(3)
evaluate_model(knn_classifier, X_tf, Y)

# svc classifier
print("SVC")
svc_classifier =  SVC(kernel="linear", C=0.025)
evaluate_model(svc_classifier, X_tf, Y)


# decision tree classifier
print("DECISION TREE CLASSIFIER")
decision_classifier = DecisionTreeClassifier(max_depth=5)
evaluate_model(decision_classifier, X_tf, Y)

# adaboost classifier
print("ADABOOST CLASSIFIER")
abc = AdaBoostClassifier(n_estimators=100, random_state=0)
evaluate_model(abc, X_tf, Y)

# gaussian naiye bayes
print("GAUSSIAN NAIVE BAYES CLASSIFIER")
gnb = GaussianNB()
evaluate_model(gnb, X_tf, Y)

# logistic regression
print("LOGISTIC REGRESSION CLASSIFIER")
logistic_regression = LogisticRegression(random_state=0)
evaluate_model(logistic_regression, X_tf, Y)

# mlp classifier
print("MLP CLASSIFIER")
mlp = MLPClassifier(random_state=1, max_iter=200)
evaluate_model(mlp, X_tf, Y)

# lda
print("LDA")
lda = LinearDiscriminantAnalysis()
evaluate_model(lda, X_tf, Y)

# random forest
print("RANDOM FOREST")
random_forest = RandomForestClassifier(max_depth=2, random_state=0)
evaluate_model(random_forest, X_tf, Y)

# # DISPLAY TABLE
# head = ["F1-score"]
# print(tabulate(f1_scores, headers=head, tablefmt="grid"))

f1_df.insert(loc=1,
          column='TF_IDF',
          value=f1_scores)
acc_df.insert(loc=1,
          column='TF_IDF',
          value=acc_scores)

NAIVE BAYES CLASSIFIER
KNN CLASSIFIER
SVC
DECISION TREE CLASSIFIER
ADABOOST CLASSIFIER
GAUSSIAN NAIVE BAYES CLASSIFIER
LOGISTIC REGRESSION CLASSIFIER
MLP CLASSIFIER
LDA
RANDOM FOREST


In [42]:
acc_df
f1_df

,Model,TF_IDF
0,Naive Bayes,68.848495
1,KNN Classifier,23.215488
2,SVC,37.092553
3,Decision tree,58.850136
4,Adaboost,65.147577
5,GNB,NaN
6,Logistic regression,67.682325
7,MLP,65.097303
8,LDA,NaN
9,Random forest,37.353747


In [43]:
#tfidf n=trigram
vectorizer = CountVectorizer(ngram_range = (3,3))
Xtf_trigram = vectorizer.fit_transform(X_final) 

In [44]:
f1_scores = []
acc_scores = []

#naive bayes classifier
print("NAIVE BAYES CLASSIFIER")
naive_bayes_classifier = MultinomialNB()
evaluate_model(naive_bayes_classifier, Xtf_trigram, Y)

# knn classifier
print("KNN CLASSIFIER")
knn_classifier = KNeighborsClassifier(3)
evaluate_model(knn_classifier, Xtf_trigram, Y)

# svc classifier
print("SVC")
svc_classifier =  SVC(kernel="linear", C=0.025)
evaluate_model(svc_classifier, Xtf_trigram, Y)


# decision tree classifier
print("DECISION TREE CLASSIFIER")
decision_classifier = DecisionTreeClassifier(max_depth=5)
evaluate_model(decision_classifier, Xtf_trigram, Y)

# adaboost classifier
print("ADABOOST CLASSIFIER")
abc = AdaBoostClassifier(n_estimators=100, random_state=0)
evaluate_model(abc, Xtf_trigram, Y)

# gaussian naiye bayes
print("GAUSSIAN NAIVE BAYES CLASSIFIER")
gnb = GaussianNB()
evaluate_model(gnb, Xtf_trigram, Y)

# logistic regression
print("LOGISTIC REGRESSION CLASSIFIER")
logistic_regression = LogisticRegression(random_state=0)
evaluate_model(logistic_regression, Xtf_trigram, Y)

# mlp classifier
print("MLP CLASSIFIER")
mlp = MLPClassifier(random_state=1, max_iter=200)
evaluate_model(mlp, X_tf, Y)

# lda
print("LDA")
lda = LinearDiscriminantAnalysis()
evaluate_model(lda, Xtf_trigram, Y)

# random forest
print("RANDOM FOREST")
random_forest = RandomForestClassifier(max_depth=2, random_state=0)
evaluate_model(random_forest, Xtf_trigram, Y)

f1_df.insert(loc=2,
          column='TF_IDF (n=3)',
          value=f1_scores)

acc_df.insert(loc=2,
          column='TF_IDF (n=3)',
          value=acc_scores)

NAIVE BAYES CLASSIFIER
KNN CLASSIFIER
SVC
DECISION TREE CLASSIFIER
ADABOOST CLASSIFIER
GAUSSIAN NAIVE BAYES CLASSIFIER
LOGISTIC REGRESSION CLASSIFIER
MLP CLASSIFIER
LDA
RANDOM FOREST


In [46]:
acc_df
f1_df

,Model,TF_IDF,TF_IDF (n=3)
0,Naive Bayes,68.848495,40.598256
1,KNN Classifier,23.215488,22.134810
2,SVC,37.092553,37.092553
3,Decision tree,58.850136,37.873553
4,Adaboost,65.147577,37.873553
5,GNB,NaN,NaN
6,Logistic regression,67.682325,39.291570
7,MLP,65.097303,65.097303
8,LDA,NaN,NaN
9,Random forest,37.353747,37.092553


In [47]:
#tfidf n=bigram
vectorizer = CountVectorizer(ngram_range = (2,2))
Xtf_bigram = vectorizer.fit_transform(X_final) 

In [48]:
f1_scores = []
acc_scores = []

#naive bayes classifier
print("NAIVE BAYES CLASSIFIER")
naive_bayes_classifier = MultinomialNB()
evaluate_model(naive_bayes_classifier, Xtf_bigram, Y)

# knn classifier
print("KNN CLASSIFIER")
knn_classifier = KNeighborsClassifier(3)
evaluate_model(knn_classifier, Xtf_bigram, Y)

# svc classifier
print("SVC")
svc_classifier =  SVC(kernel="linear", C=0.025)
evaluate_model(svc_classifier, Xtf_bigram, Y)


# decision tree classifier
print("DECISION TREE CLASSIFIER")
decision_classifier = DecisionTreeClassifier(max_depth=5)
evaluate_model(decision_classifier, Xtf_bigram, Y)

# adaboost classifier
print("ADABOOST CLASSIFIER")
abc = AdaBoostClassifier(n_estimators=100, random_state=0)
evaluate_model(abc, Xtf_bigram, Y)

# gaussian naiye bayes
print("GAUSSIAN NAIVE BAYES CLASSIFIER")
gnb = GaussianNB()
evaluate_model(gnb, Xtf_bigram, Y)

# logistic regression
print("LOGISTIC REGRESSION CLASSIFIER")
logistic_regression = LogisticRegression(random_state=0)
evaluate_model(logistic_regression, Xtf_bigram, Y)

# mlp classifier
print("MLP CLASSIFIER")
mlp = MLPClassifier(random_state=1, max_iter=200)
evaluate_model(mlp, X_tf, Y)

# lda
print("LDA")
lda = LinearDiscriminantAnalysis()
evaluate_model(lda, Xtf_bigram, Y)

# random forest
print("RANDOM FOREST")
random_forest = RandomForestClassifier(max_depth=2, random_state=0)
evaluate_model(random_forest, Xtf_bigram, Y)

# DISPLAY TABLE
f1_df.insert(loc=3,
          column='TF_IDF (n=2)',
          value=f1_scores)
acc_df.insert(loc=3,
          column='TF_IDF (n=2)',
          value=acc_scores)

NAIVE BAYES CLASSIFIER
KNN CLASSIFIER
SVC
DECISION TREE CLASSIFIER
ADABOOST CLASSIFIER
GAUSSIAN NAIVE BAYES CLASSIFIER
LOGISTIC REGRESSION CLASSIFIER
MLP CLASSIFIER
LDA
RANDOM FOREST


In [49]:
# fasttext
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz"

--2022-10-25 09:12:45--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  35.8MB/s    in 2m 8s   

2022-10-25 09:14:54 (33.5 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [50]:
fasttext.util.download_model('en', if_exists='ignore') # english
T_ft = fasttext.load_model('cc.en.300.bin')

In [51]:
X_fasttext = [T_ft.get_word_vector(words) for words in X_final] 

In [52]:
f1_scores = []
acc_scores = []

#naive bayes classifier
print("NAIVE BAYES CLASSIFIER")
naive_bayes_classifier = MultinomialNB()
evaluate_model(naive_bayes_classifier, X_fasttext, Y)

# knn classifier
print("KNN CLASSIFIER")
knn_classifier = KNeighborsClassifier(3)
evaluate_model(knn_classifier, X_fasttext, Y)

# svc classifier
print("SVC")
svc_classifier =  SVC(kernel="linear", C=0.025)
evaluate_model(svc_classifier, X_fasttext, Y)


# decision tree classifier
print("DECISION TREE CLASSIFIER")
decision_classifier = DecisionTreeClassifier(max_depth=5)
evaluate_model(decision_classifier, X_fasttext, Y)

# adaboost classifier
print("ADABOOST CLASSIFIER")
abc = AdaBoostClassifier(n_estimators=100, random_state=0)
evaluate_model(abc, X_fasttext, Y)

# gaussian naiye bayes
print("GAUSSIAN NAIVE BAYES CLASSIFIER")
gnb = GaussianNB()
evaluate_model(gnb, X_fasttext, Y)

# logistic regression
print("LOGISTIC REGRESSION CLASSIFIER")
logistic_regression = LogisticRegression(random_state=0)
evaluate_model(logistic_regression, X_fasttext, Y)

# mlp classifier
print("MLP CLASSIFIER")
mlp = MLPClassifier(random_state=1, max_iter=200)
evaluate_model(mlp, X_tf, Y)

# lda
print("LDA")
lda = LinearDiscriminantAnalysis()
evaluate_model(lda, X_fasttext, Y)

# random forest
print("RANDOM FOREST")
random_forest = RandomForestClassifier(max_depth=2, random_state=0)
evaluate_model(random_forest, X_fasttext, Y)

# DISPLAY TABLE
f1_df.insert(loc=4,
          column='Fasttext',
          value=f1_scores)
acc_df.insert(loc=4,
          column='Fasttext',
          value=acc_scores)

NAIVE BAYES CLASSIFIER
KNN CLASSIFIER
SVC
DECISION TREE CLASSIFIER
ADABOOST CLASSIFIER
GAUSSIAN NAIVE BAYES CLASSIFIER
LOGISTIC REGRESSION CLASSIFIER
MLP CLASSIFIER
LDA
RANDOM FOREST


In [53]:
f1_df

,Model,TF_IDF,TF_IDF (n=3),TF_IDF (n=2),Fasttext
0,Naive Bayes,68.848495,40.598256,50.478347,NaN
1,KNN Classifier,23.215488,22.134810,22.134810,40.794511
2,SVC,37.092553,37.092553,37.092553,37.092553
3,Decision tree,58.850136,37.873553,39.149575,47.878934
4,Adaboost,65.147577,37.873553,41.629820,50.456436
5,GNB,NaN,NaN,NaN,26.572284
6,Logistic regression,67.682325,39.291570,53.296998,45.495085
7,MLP,65.097303,65.097303,65.097303,65.097303
8,LDA,NaN,NaN,NaN,52.877024
9,Random forest,37.353747,37.092553,37.092553,38.074604


In [54]:
acc_df

,Model,TF_IDF,TF_IDF (n=3),TF_IDF (n=2),Fasttext
0,Naive Bayes,50.921338,36.991471,45.102300,NaN
1,KNN Classifier,33.563967,33.333333,33.333333,36.968194
2,SVC,33.333333,33.333333,33.333333,33.333333
3,Decision tree,43.710982,33.632589,34.131224,35.151357
4,Adaboost,48.996563,33.632589,35.372465,38.237483
5,GNB,NaN,NaN,NaN,36.100674
6,Logistic regression,49.546313,34.085267,40.130061,36.345725
7,MLP,51.187857,51.187857,51.187857,51.187857
8,LDA,NaN,NaN,NaN,41.200824
9,Random forest,33.433584,33.333333,33.333333,33.658801


In [56]:
f1_df.dropna()
acc_df.dropna()

,Model,TF_IDF,TF_IDF (n=3),TF_IDF (n=2),Fasttext
1,KNN Classifier,33.563967,33.333333,33.333333,36.968194
2,SVC,33.333333,33.333333,33.333333,33.333333
3,Decision tree,43.710982,33.632589,34.131224,35.151357
4,Adaboost,48.996563,33.632589,35.372465,38.237483
6,Logistic regression,49.546313,34.085267,40.130061,36.345725
7,MLP,51.187857,51.187857,51.187857,51.187857
9,Random forest,33.433584,33.333333,33.333333,33.658801


In [58]:
f1_df.to_csv('f1-scores.csv')
acc_df.to_csv('accuracy-scores.csv')

In [59]:
sentiment_data

,Comment,"Sentiment: Positive, neagtive, neutral",Clean
0,Excellent...vaa malla malla show la umdeta dum...,P,excellent vaa malla malla show la umdeta dumbu...
1,Valerie sir na voice keniji YouTube de,N,valerie sir na voice keniji youtube de
2,Firstrde bloopers thojale plz,NU,firstrde blooper thojale plz
3,Congress 100 episode mantinek niklna mata epi...,P,congress episode mantinek niklna mata episodel...
4,Err 2 jana matra aath....full team malthini ep...,N,err jana matra aath full team malthini episode
...,...,...,...
1694,Congratulations both of u.. ❤️chende da uncle ...,NU,congratulation u chende da uncle gu brother si...
1695,Teacher na principal Yana athige,NU,teacher na principal yana athige
1696,Dala unda... song launch (programme) dha video...,NU,dala unda song launch programme dha video edag...
1697,Sachin anna ❤️\nDala unda... 🍻,NU,sachin anna dala unda


In [60]:
from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

FileNotFoundError: ignored

In [ ]:
from itertools import islice
list(islice(wv.vocab, 13030, 13050))

In [ ]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        print("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train, test = train_test_split(df, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

In [ ]:
df

In [ ]:
df.dropna()

In [ ]:
df.to_csv('metrics.csv')